In [ ]:
import numpy as np

import linpde_gp
import probnum as pn
from matplotlib import pyplot as plt

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0002_poisson_dirichlet_fem"
config.target = "jmlr"
config.debug_mode = True

plt.rcParams.update(config.tueplots_bundle())

## Problem Definition

In [ ]:
domain = linpde_gp.domains.asdomain([-1.0, 1.0])

In [ ]:
# Boundary Values
g = np.asarray((0.0, 1.0))

# PDE RHS
f = linpde_gp.functions.Constant(input_shape=(), value=2.0)

# True Solution
u = linpde_gp.problems.pde.PoissonEquation1DConstRHSDirichletProblemSolution(
    domain=domain,
    rhs=f.value,
    boundary_values=g,
)

In [ ]:
bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(
    pde=linpde_gp.problems.pde.PoissonEquation(
        domain=domain,
        rhs=linpde_gp.randprocs.DeterministicProcess(f),
    ),
    boundary_values=pn.randvars.asrandvar(g),
    solution=u,
)

## Test and Trial Bases

In [ ]:
legacy_basis = linpde_gp.galerkin.bases.LinearInterpolationBasis(
    domain=bvp.pde.domain,
    num_elements=5,
)

trial_basis = linpde_gp.functions.bases.UnivariateLinearInterpolationBasis(
    legacy_basis.grid,
    zero_boundary=False,
)

test_basis = linpde_gp.functions.bases.UnivariateLinearInterpolationBasis(
    legacy_basis.grid,
    zero_boundary=True,
)

trial_proj = trial_basis.l2_projection()
test_proj = test_basis.l2_projection(normalized=False)

## Plotting

In [ ]:
%matplotlib inline

import matplotlib.axes

from linpde_gp.typing import RandomProcessLike, RandomVariableLike

plt_grid = domain.uniform_grid(200)

def plot_belief(
    ax: matplotlib.axes.Axes,
    *,
    u: pn.randprocs.GaussianProcess,
    conditioned_on: list[str] = [],
    solution: RandomProcessLike | None = None,
):
    # Solution Belief
    cond_events_str = _build_cond_events_str(conditioned_on)

    u.plot(
        ax,
        plt_grid,
        num_samples=3,
        rng=np.random.default_rng(24),
        color="C0",
        label=(
            fr"$u \mid {cond_events_str}$"
            if len(cond_events_str) > 0
            else "$u$"
        ),
    )

    # True Solution
    if solution is not None:
        linpde_gp.randprocs.asrandproc(solution).plot(
            ax,
            plt_grid,
            color="C1",
            label="$u^\star$",
        )

    ax.legend()

def plot_proj_belief(
    ax: matplotlib.axes.Axes,
    *,
    u: pn.randprocs.GaussianProcess,
    conditioned_on: list[str] = [],
    solution: RandomProcessLike | None = None,
):
    # Projected Solution Belief
    Pu = trial_proj(u)

    Pu_gp = linpde_gp.randprocs.ParametricGaussianProcess(
        weights=Pu,
        feature_fn=trial_basis,
    )

    cond_events_str = _build_cond_events_str(conditioned_on)

    Pu_gp.plot(
        ax,
        plt_grid,
        num_samples=3,
        rng=np.random.default_rng(24),
        color="C0",
        label=(
            fr"$\mathcal{{P}}[u] \mid {cond_events_str}$"
            if len(cond_events_str) > 0
            else r"$\mathcal{P}[u]$"
        ),
    )

    # True Solution
    if solution is not None:
        linpde_gp.randprocs.asrandproc(solution).plot(
            ax,
            plt_grid,
            color="C1",
            label="$u^\star$",
        )

    ax.legend()

def _build_cond_events_str(conditioned_on: list[str]) -> str:
    events = []

    for key in conditioned_on:
        match key:
            case "bc":
                events.append(r"\mathrm{BC}")
            case "bc_fem":
                events.append(r"\mathrm{BC}_\mathrm{FEM}")
            case "pde":
                events.append(r"\mathrm{PDE}")
            case _:
                raise ValueError(f"Unknown event '{key}'")
    
    return ", ".join(events)

In [ ]:
evals = trial_basis(plt_grid)

plt.plot(plt_grid, evals[...])

# for x in trial_basis.grid:
#     plt.axvline(x)

plt.show()

## Prior

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=pn.functions.Zero(
        input_shape=(),
        output_shape=(),
    ),
    cov=linpde_gp.randprocs.kernels.GalerkinKernel(
        pn.randprocs.kernels.Matern(
            input_shape=(),
            lengthscale=1.0,
            nu=1.5,
        ),
        projection=trial_proj,
    ),
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_prior,
    solution=bvp.solution,
)

experiment_utils.savefig("00_u_prior")

In [ ]:
plot_proj_belief(
    plt.gca(),
    u=u_prior,
    solution=bvp.solution,
)

experiment_utils.savefig("00_u_prior_proj")

## Posterior

### Conditioning on the Boundary Conditions

In [ ]:
X_bc = np.asarray(bvp.boundary_conditions[0].boundary)
Y_bc = bvp.boundary_conditions[0].values

u_cond_bc = u_prior.condition_on_observations(
    np.zeros_like(Y_bc.mean),
    X=X_bc,
    b=-Y_bc
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc"],
    solution=bvp.solution,
)

experiment_utils.savefig("01_u_cond_bc")

In [ ]:
plot_proj_belief(
    plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc"],
    solution=bvp.solution,
)

experiment_utils.savefig("01_u_cond_bc_proj")

In [ ]:
A_bc = np.eye(len(trial_basis))[[0, -1], :]

u_cond_bc = u_cond_bc.condition_on_observations(
    Y_bc.mean,
    L=A_bc @ trial_proj,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc", "bc_fem"],
    solution=bvp.solution,
)

experiment_utils.savefig("02_u_cond_bc_bc_fem")

In [ ]:
plot_proj_belief(
    plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc", "bc_fem"],
    solution=bvp.solution,
)

experiment_utils.savefig("02_u_cond_bc_bc_fem_proj")

### Conditioning on the PDE

In [ ]:
diffop_galerkin = bvp.pde.diffop.weak_form(test_basis)(trial_basis)
rhs_galerkin = test_proj(bvp.pde.rhs)

In [ ]:
diffop_galerkin.todense()

In [ ]:
rhs_galerkin.mean

In [ ]:
u_cond_bc_pde = u_cond_bc.condition_on_observations(
    np.zeros_like(rhs_galerkin.mean),
    L=diffop_galerkin @ trial_proj,
    b=-rhs_galerkin,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde,
    conditioned_on=["bc", "bc_fem", "pde"],
    solution=bvp.solution,
)

experiment_utils.savefig("03_u_cond_bc_bc_fem_pde")

In [ ]:
plot_proj_belief(
    plt.gca(),
    u=u_cond_bc_pde,
    conditioned_on=["bc", "bc_fem", "pde"],
    solution=bvp.solution,
)

experiment_utils.savefig("03_u_cond_bc_bc_fem_pde_proj")

### Testing

In [ ]:
import scipy.integrate

phis_grid = trial_basis(plt_grid)

def sample_trial_proj(sample):
    res = scipy.integrate.trapezoid(
        phis_grid * sample[..., None],
        plt_grid,
        axis=0,
    )

    return trial_proj.normalizer(res)

samples = np.stack(
    [
        sample - np.sum(sample_trial_proj(sample) * phis_grid, axis=-1)
        for sample in u_cond_bc.sample(np.random.default_rng(24), plt_grid, size=(2000,))
    ],
    axis=-1,
)

samples_std = np.std(samples, axis=-1)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde,
    conditioned_on=["bc", "bc_fem", "pde"],
    solution=bvp.solution,
)

post_mean = u_cond_bc_pde.mean(plt_grid)

plt.plot(plt_grid, post_mean + 1.96 * samples_std, c="C3", label="Sample Cred. Int.")
plt.plot(plt_grid, post_mean - 1.96 * samples_std, c="C3")

plt.legend()

experiment_utils.savefig("posterior_std_samples")

## Conditioning on Superconvergence at the Nodes

In [ ]:
u_cond_fem = u_cond_bc_pde.condition_on_observations(
    u_cond_bc_pde.mean(trial_basis.grid[2:-2]),
    X=trial_basis.grid[2:-2],
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_fem,
    conditioned_on=["bc", "bc_fem", "pde"],
    solution=bvp.solution,
)

experiment_utils.savefig("04_u_cond_bc_bc_fem_pde_superconv")